# Statistical Comparisons Generator

This notebook separates different measurement categories into numerous populations and compares the statistical significance of data trends

In [1]:
#all package imports needed for notebook here
import pandas as pd
import numpy as np

%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
from scipy.stats import kruskal
import seaborn as sns
import numpy as np
import pprint as pp
from datetime import datetime
from IPython.display import display
import scipy.stats as stats
import numpy as np
from numpy.fft import fft,ifft,fft2,ifft2,fftshift

df2013 = pd.read_csv('Kwadella_winter_2013_cleaned.csv')
df2014 = pd.read_csv('Kwadella_winter_2014_cleaned.csv')

/home/mackenzie/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/mackenzie/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


First we're looking at an analysis of variance between 2014 and 2013 to see how significantly the years differed. We'll look at how the outdoor temperatures varied (as a soft control), then compare the year's pollutant concentrations  to get a read for how well the intervention reduced pollution. We'll do this again with diurnal plots to be sure. 

Cross correlation will let us track dusttraks vs PM2.5 vs PM10 vs CO concentrations and traking the rise and fall of each pollutant over the course of the entire winter. We're starting by just looking at the entire winter, then we may divide pollution measurements based on temperature deviation of a given day or by month

ANOVA is best for comparing one category of measurement at different time points to see if there is significant variance between those groups. for example: 1 way ANOVA done on dusttraks from the mean temp track, dusttraks from 1 sd colder than the mean, and dusttraks from 1 sd warmer than the mean. This measurement would tell us if there is variance in pollution between the three groups. This test could also be applied to 2013 vs 2014 dusttraks to find statistical significance in the differences in pollution before and after intervention

In [2]:
def get_datetime(s):
    """strips date and time from the already existing date column"""
    dt = datetime.strptime(s, "%m/%d/%y %H:%M")
    return dt

def daysSinceStart(df):
    """get time since epoch using a series for month and day
    takes in dataframe and returns the dataframe with an added colum for days since the beginning of data collection"""
    dayArray = np.array(df.Day)
    monthArray = np.array(df.Month)
    
    runningDays = []
    for day, month in zip(dayArray, monthArray):
        if month == 7:
            total_days = 0
        elif month == 8:
            total_days = 31 
        elif month == 9:
            total_days = 61
        else:
            raise ValueError 
        total_days = total_days + day
        runningDays.append(total_days)
    df['DayCount'] = pd.Series(runningDays, index = df.index)
    return df

#Daily Average Temp
def compute_avg_temp(df):
    df['DailyAverageTemp'] = df['Temperature_(degC)'].mean()
    return df

def preprocess(df):
    """runs the datetime and daysSinceStart helper functions
    takes and returns a dataframe"""
    res = df.copy()
    datetimes = res.Date.apply(get_datetime)
    res['Hour'] = datetimes.apply(lambda dt: dt.hour)
    res['Day'] = datetimes.apply(lambda dt: dt.day)
    res['Month'] = datetimes.apply(lambda dt: dt.month)
    res['Year'] = datetimes.apply(lambda dt: dt.year)
    return res




In [3]:
df2013 = preprocess(df2013)
df2013 = daysSinceStart(df2013)
grouped = df2013.groupby('DayCount')
df2013 = grouped.apply(compute_avg_temp)
df2013 = compute_avg_temp(df2013)
df2013['Dusttraks_(mg/m3)']= df2013['Dusttraks_(mg/m3)']*0.14


df2014 = daysSinceStart(df2014)
grouped = df2014.groupby('DayCount')
df2014 = grouped.apply(compute_avg_temp)
df2014 = compute_avg_temp(df2014)
df2014['Dusttraks_(mg/m3)']= df2014['Dusttraks_(mg/m3)']*0.14

In [4]:
kruskal(df2013['Temperature_(degC)'],df2014['Temperature_(degC)'])


KruskalResult(statistic=2365.9088083635561, pvalue=0.0)

In [5]:
def cross_correlation_using_fft(x, y):
    f1 = fft(x)
    f2 = fft(np.flipud(y))
    cc = np.real(ifft(f1 * f2))
    return fftshift(cc)
 
# shift &lt; 0 means that y starts 'shift' time steps before x # shift &gt; 0 means that y starts 'shift' time steps after x
def compute_shift(x, y):
    assert len(x) == len(y)
    c = cross_correlation_using_fft(x, y)
    assert len(c) == len(x)
    zero_index = int(len(x) / 2) - 1
    shift = zero_index - np.argmax(c)
    return shift

In [6]:
compute_shift(df2013['Temperature_(degC)'], df2013['Dusttraks_(mg/m3)'])

62676

In [7]:
np.correlate(df2013['Temperature_(degC)'], df2013['Dusttraks_(mg/m3)'])

array([ nan])

In [14]:
df2013.head()


,Unnamed: 0,Dusttraks_(mg/m3),PM10_(ug/m3),PM2.5_(ug/m3),SO2_(ppb),CO_(ppm),NO_(ppb),NO2_(ppb),Date,Temperature_(degC),...,H15-K-S,H17-K-N,H18-K-S,H20-K-S,Hour,Day,Month,Year,DayCount,DailyAverageTemp
0,120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/2/13 14:00,NaN,...,16,15.5,16.5,15.0,14,2,7,2013,2,10.798239
1,121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/2/13 14:01,NaN,...,16,15.5,16.5,15.1,14,2,7,2013,2,10.798239
2,122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/2/13 14:02,NaN,...,16,15.5,16.5,15.1,14,2,7,2013,2,10.798239
3,123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/2/13 14:03,NaN,...,16,15.5,16.5,15.2,14,2,7,2013,2,10.798239
4,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/2/13 14:04,NaN,...,16,15.5,16.5,15.2,14,2,7,2013,2,10.798239


In [37]:
def normalize(df, headers):
    normdf = df[headers]
    for h in headers:
        means = pd.rolling_mean(normdf[h],3)
        print means
        normdf.loc[:,h] = normdf[h].fillna(means)
    df_norm = (normdf-normdf.mean())/(normdf.max() - normdf.min())
    return df_norm

dfnorm13 = normalize(df2013, ['Temperature_(degC)', 'Dusttraks_(mg/m3)'])

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
26       NaN
27       NaN
28       NaN
29       NaN
          ..
125325   NaN
125326   NaN
125327   NaN
125328   NaN
125329   NaN
125330   NaN
125331   NaN
125332   NaN
125333   NaN
125334   NaN
125335   NaN
125336   NaN
125337   NaN
125338   NaN
125339   NaN
125340   NaN
125341   NaN
125342   NaN
125343   NaN
125344   NaN
125345   NaN
125346   NaN
125347   NaN
125348   NaN
125349   NaN
125350   NaN
125351   NaN
125352   NaN
125353   NaN
125354   NaN
Name: Temperature_(degC), dtype: float64
0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       N

In [33]:
#need to replace nans with a rolling mean, then normalize, then correlate 



,Temperature_(degC),Dusttraks_(mg/m3)
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [30]:
np.correlate(dfnorm13['Temperature_(degC)'],dfnorm13['Dusttraks_(mg/m3)'], mode = 'full')

array([ nan,  nan,  nan, ...,  nan,  nan,  nan])

In [26]:
np.corrcoef(dfnorm13['Temperature_(degC)'],dfnorm13['Dusttraks_(mg/m3)'])

array([[ nan,  nan],
       [ nan,  nan]])